# Automated Report Production in Python

In [ ]:
%matplotlib agg

import datetime
import math
from matplotlib import rcParams
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
fig = plt.figure()
fig.set_figheight(12.75)
fig.set_figwidth(21)
fig.set_facecolor('white')
rcParams['font.family'] = 'Calibri'

In [ ]:
# Add explanation: 16 rows + 1 header + 1 footer = 18
header =  plt.subplot2grid((18, 8), (0, 0), rowspan=1, colspan=8)
chart_1 = plt.subplot2grid((18, 8), (1, 0), rowspan=8, colspan=4)
chart_2 = plt.subplot2grid((18, 8), (1, 4), rowspan=8, colspan=4)
chart_3 = plt.subplot2grid((18, 8), (9, 0), rowspan=8, colspan=4)
chart_4 = plt.subplot2grid((18, 8), (9, 4), rowspan=8, colspan=4)
footer =  plt.subplot2grid((18, 8), (17, 0), rowspan=1, colspan=8)

In [ ]:
display(fig)

In [ ]:
# Get picture
def get_pic(file, resize_factor=1):
    pic = Image.open(file)
    width  = math.floor(pic.size[0] / resize_factor)
    height = math.floor(pic.size[1] / resize_factor)
    pic = pic.resize((width, height), resample=Image.ANTIALIAS)
    return pic, width, height

In [ ]:
# Add FIPS
# Top left
csps, width, height = get_pic('FIPs/csps.png', resize_factor=2.5)
fig.figimage(csps, xo=0, yo=fig.bbox.ymax - height, origin='upper', zorder=1)

# Bottom left
my_school, width, height = get_pic('FIPs/my_school.png', resize_factor=1.2)
fig.figimage(my_school, xo=0, yo=0, origin='upper', zorder=1)

# Bottom right
canada, width, height = get_pic('FIPs/canada.png', resize_factor=2.5)
fig.figimage(canada, xo=fig.bbox.xmax - width, yo=0, origin='upper', zorder=1)

In [ ]:
fig.tight_layout(pad=1.0, h_pad=1.8)
display(fig)

In [ ]:
# Add text
footer.text(x=0.9, y=0.5, horizontalalignment='right', fontsize=8, s='See Appendix 1 for Methodology – ' + \
    'Consulter l\'Annexe 1 pour la méthodologie')

footer.text(x=0.9, y=-0.4, horizontalalignment='right', fontsize=8, s='Report generated on – ' + \
    'Rapport généré le ' + datetime.datetime.today().strftime('%d/%m/%Y'))

footer.text(x=0.9, y=-1.3, horizontalalignment='right', fontsize=8, s='Page 1/1')

In [ ]:
display(fig)

In [ ]:
def remove_lines(ax):
    ax.axis('off')

remove_lines(header)
remove_lines(footer)

In [ ]:
display(fig)

In [ ]:
# Add pie chart
# Example with fake data
values = [4, 5, 6]
labels = ['Moose', 'Beaver', 'Chipmunk']

chart_1.pie(x=values, labels=labels, explode=[0, 0.05, 0])
# Fix skew
chart_1.axis('equal')

In [ ]:
display(fig)

In [ ]:
# Now let's import data and use that for the other 3 charts
import pandas as pd

In [ ]:
store_data = pd.read_csv('store_data.csv', sep=',', index_col=0, encoding='utf-8')

In [ ]:
# Any database people here, it's denormalized
# Can do joins in Pandas if needed
store_data.head()

In [ ]:
# Coffee sales per month, with title
monthly_coffee = store_data.loc[store_data['product_name'] == 'Coffee', ['quantity', 'month', 'month_num']]
monthly_coffee = monthly_coffee.groupby(['month', 'month_num'], as_index=False).sum()
monthly_coffee = monthly_coffee.sort_values('month_num')

In [ ]:
monthly_coffee

In [ ]:
# Plot the data
chart_2.plot(monthly_coffee['month'], monthly_coffee['quantity'])

In [ ]:
# Add a title
chart_2.set_title('Monthly Coffee Sales', fontsize=16)

In [ ]:
display(fig)

In [ ]:
# Chart 3: Bar chart percent satisfied by product
store_data.head()

In [ ]:
satisfied = store_data.loc[:, ['product_name', 'satisfied']].groupby(['product_name'], as_index=False).sum()
total = store_data.loc[:, ['product_name', 'satisfied']].groupby(['product_name'], as_index=False).count()
percentages = satisfied['satisfied'] / total['satisfied']

In [ ]:
chart_3.set_ylim(0, 1.1)
vals = chart_3.get_yticks()
chart_3.set_yticklabels(['{:,.0%}'.format(x) for x in vals])

chart_3.bar(satisfied['product_name'], height=percentages, )
chart_3.set_title('Customer Satisfaction by Product', fontsize=16)

In [ ]:
display(fig)

In [ ]:
# Chart 4: pie chart for market share of products
counts = store_data['product_name'].value_counts()

In [ ]:
chart_4.pie(x=counts.values, labels=counts.index)
chart_4.set_title('Market Share by Product', fontsize=16)
# Fix skew
chart_4.axis('equal')

In [ ]:
display(fig)

In [ ]:
# Save as PDF
fig.savefig('Big Report - Grand rapport.pdf')

In [ ]:
# Add for loop, show audience can extend this to generate many, many reports at once

### 1. Import packages
* Pandas: Import, crunch raw data
* Matplotlib: Create charts
* PIL: Image processing (use to add FIPs to the PDFs)

### 2. Import raw data
#### Tip: Ensure encodings match to prevent Français -> FranÃ§ais

### 3. Build layout

#### Using aspect ratio of a legal landscape document

#### Divide space for our 4 charts
#### Explanation:
* Divide page into a 16x8 grid
* Assign each chart to a variable and allocate it space
* E.g. the first chart is assigned to ax0, is given 8 rows and 4 columns worth of space, starting at the top left corner, also known as position (0, 0)